#### 📌 **HİPERPARAMETRELERİ BURADAN AYARLA**

In [1]:
learning_rate = 1e-3       
batch_size = 32           
pca_variance = 0.97       # PCA varyans koruma oranı
roi_crop_size = (256,256)
patch_size = (128,128)
mlp_hidden_units = [128, 256,512]
mlp_dropout_rate = 0.55
mlp_epochs = 55
mlp_earlystop_patience = 9
pixels_per_cell=(16,16)
cells_per_block=(2,2)
orientations=10

In [2]:
import os, re, joblib
import cv2
import numpy as np
import pandas as pd
from skimage.feature import hog, greycomatrix, greycoprops
from skimage.filters import threshold_otsu, median
from skimage.morphology import opening, disk
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import classification_report, f1_score, accuracy_score, precision_score, recall_score, roc_auc_score
from joblib import Parallel, delayed
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [3]:
def process_single_image(args):
    img_path, label, classes = args
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, roi_crop_size)
    img = img.astype(np.float32)

    center = np.array(roi_crop_size) // 2
    ps = np.array(patch_size) // 2
    patch = img[center[0]-ps[0]:center[0]+ps[0], center[1]-ps[1]+0:center[1]+ps[1]]

    patch = median(patch)
    patch = cv2.GaussianBlur(patch, (3, 3), 0)
    patch = patch.astype(np.float32) / 255.0

    return patch, classes.index(label)


In [4]:

def extract_and_preprocess_parallel(root_dir, classes, n_jobs=-1):
    pattern = re.compile(r'^[A-Z]_\d+_\d+\.(LEFT|RIGHT)_(CC|MLO)\.jpg$', re.IGNORECASE)
    siniflar = ['benign', 'cancer', 'normal']
    yollar, etiketler = [], []

    for cls in siniflar:
        cls_yolu = os.path.join(root_dir, cls)
        if not os.path.isdir(cls_yolu):
            continue
        for durum_id in os.listdir(cls_yolu):
            durum_yolu = os.path.join(cls_yolu, durum_id)
            if not os.path.isdir(durum_yolu):
                continue
            for dosya_adi in os.listdir(durum_yolu):
                if pattern.match(dosya_adi):
                    yollar.append(os.path.join(durum_yolu, dosya_adi))
                    etiketler.append(cls)

    # === Paralel işleme ===
    arg_list = [(img_path, lbl, classes) for img_path, lbl in zip(yollar, etiketler)]
    results = Parallel(n_jobs=n_jobs)(delayed(process_single_image)(args) for args in arg_list)

    X, y = zip(*results)
    return np.array(X), np.array(y)

In [5]:
def extract_features_from_image(im):
    h = hog(im, pixels_per_cell=pixels_per_cell,
                cells_per_block=cells_per_block,
                orientations=orientations)
    g = greycomatrix((im*255).astype(np.uint8), [1], [0], levels=256)
    glcm = [greycoprops(g, prop)[0, 0] for prop in ('contrast', 'homogeneity', 'energy')]
    area = np.sum(im > 0.5)
    perimeter = np.sum(cv2.Canny((im * 255).astype(np.uint8), 50, 150) > 0)
    ecc = 0
    if area > 10:
        pts = np.argwhere(im > 0.5)
        rect = cv2.minAreaRect(pts.astype(np.float32))
        ecc = rect[1][0] / (rect[1][1] + 1e-8)
    return np.hstack([h, glcm, area, perimeter, ecc])

def extract_features(X, out_prefix=None, n_jobs=-1):
    # 2. UPDATE THE FUNCTION NAME INSIDE THE PARALLEL CALL HERE
    feats = Parallel(n_jobs=n_jobs)(delayed(extract_features_from_image)(im) for im in X)
    feats = np.array(feats)
    pca = PCA(n_components=pca_variance)
    X_feat = pca.fit_transform(feats)
    return X_feat, pca

 Metirkler**

In [6]:
if __name__ == '__main__':
    classes = ['normal','benign','cancer']
    kok_dizin = os.path.join(os.getcwd(), "Project1")
    X_test, y_test = extract_and_preprocess_parallel(kok_dizin, classes,n_jobs=-1)
    X_feat, pca = extract_features(X_test)
    model = load_model("MLP_1_3.h5")
    T_m = 0.22
    T_b = 0.64
    probs = model.predict(X_feat)
    preds = [2 if p[2] >= T_m else 1 if p[1] >= T_b else 0 for p in probs]
    print("\n --------------MLP RESULTS----------------")
    print(classification_report(y_test, preds, target_names=classes))
    print("Confusion Matrix:\n", confusion_matrix(y_test, preds))
    print("Accuracy:", accuracy_score(y_test, preds))
    print("Precision:", precision_score(y_test, preds, average='macro'))
    print("Recall:", recall_score(y_test, preds, average='macro'))
    print("📌 F1 Score:", f1_score(y_test, preds, average='macro'))
    try:
        print("AUC:", roc_auc_score(pd.get_dummies(y_test), probs,
                                 multi_class='ovr', average='macro'))
    except:
        print("AUC hesaplanamadı.")
    


 --------------MLP RESULTS----------------
              precision    recall  f1-score   support

      normal       0.45      0.54      0.49        80
      benign       0.38      0.69      0.49        80
      cancer       1.00      0.01      0.02        80

    accuracy                           0.41       240
   macro avg       0.61      0.41      0.34       240
weighted avg       0.61      0.41      0.34       240

Confusion Matrix:
 [[43 37  0]
 [25 55  0]
 [27 52  1]]
Accuracy: 0.4125
Precision: 0.6115253411306043
Recall: 0.41250000000000003
📌 F1 Score: 0.33573045267489715
AUC: 0.6147265624999999


In [7]:
model = joblib.load("RF_1_3.pkl")
T_m = 0.46
T_b = 0.43
classes = ['normal', 'benign', 'cancer']
label_map = {'normal': 0, 'benign': 1, 'cancer': 2}
probs = model.predict_proba(X_feat) 
preds = [2 if p[2] >= T_m else 1 if p[1] >= T_b else 0 for p in probs]
print("\n --------------RF RESULTS----------------")
print(classification_report(y_test, preds, target_names=classes))
print("Confusion Matrix:\n", confusion_matrix(y_test, preds))
print("Accuracy:", accuracy_score(y_test, preds))
print("Precision:", precision_score(y_test, preds, average='macro'))
print("Recall:", recall_score(y_test, preds, average='macro'))
print("📌 F1 Score:", f1_score(y_test, preds, average='macro'))
try:
    print("AUC:", roc_auc_score(pd.get_dummies(y_test), probs,
                                 multi_class='ovr', average='macro'))
except:
    print("AUC hesaplanamadı.")


 --------------RF RESULTS----------------
              precision    recall  f1-score   support

      normal       0.35      0.57      0.43        80
      benign       0.40      0.24      0.30        80
      cancer       0.37      0.28      0.31        80

    accuracy                           0.36       240
   macro avg       0.37      0.36      0.35       240
weighted avg       0.37      0.36      0.35       240

Confusion Matrix:
 [[46 11 23]
 [46 19 15]
 [41 17 22]]
Accuracy: 0.3625
Precision: 0.37226221582324603
Recall: 0.3625
📌 F1 Score: 0.348474398446673
AUC: 0.5496354166666667


In [8]:
loaded_xgb_model_booster = xgb.Booster() 
loaded_xgb_model_booster.load_model("XGBoost_1_3.json") 
T_m = 0.37
T_b = 0.37
classes = ['normal', 'benign', 'cancer']
label_map = {'normal': 0, 'benign': 1, 'cancer': 2}


dtest_predict = xgb.DMatrix(X_feat) 
probs = loaded_xgb_model_booster.predict(dtest_predict) 
preds = [2 if p[2] >= T_m else 1 if p[1] >= T_b else 0 for p in probs]
T_m = 0.40
T_b = 0.40

print("\n --------------XP-B RESULTS----------------")
print(classification_report(y_test, preds, target_names=classes))
print("Confusion Matrix:\n", confusion_matrix(y_test, preds))
print("Accuracy:", accuracy_score(y_test, preds))
print("Precision:", precision_score(y_test, preds, average='macro'))
print("Recall:", recall_score(y_test, preds, average='macro'))
print("📌 F1 Score:", f1_score(y_test, preds, average='macro'))
try:
    print("AUC:", roc_auc_score(pd.get_dummies(y_test), probs,
                                 multi_class='ovr', average='macro'))
except:
    print("AUC hesaplanamadı.")


 --------------XP-B RESULTS----------------
              precision    recall  f1-score   support

      normal       0.46      0.47      0.47        80
      benign       0.32      0.15      0.21        80
      cancer       0.33      0.50      0.40        80

    accuracy                           0.38       240
   macro avg       0.37      0.38      0.36       240
weighted avg       0.37      0.38      0.36       240

Confusion Matrix:
 [[38 10 32]
 [19 12 49]
 [25 15 40]]
Accuracy: 0.375
Precision: 0.37277249028912
Recall: 0.375
📌 F1 Score: 0.3574246526153657
AUC: 0.5552604166666667
